# Selenium이용한 크롤링실습

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
service = Service(executable_path=ChromeDriverManager().install())

## 싼 주유소 찾기 Opinet

- https://www.opinet.co.kr/user/main/mainView.do

### 서울시 구별 주유소 가격 정도 얻어오기

- 사이트에서 [싼주유소찾기] > [지역별] 메뉴 선택
- 지역은 '서울' > 원하는 구 지정
    - '시/군/구'를 선택하는 select option의 XPath 찾기

- '조회'버튼 클릭
- 조회된 주유소 정보 '엑셀저장' 버튼 클릭하여 엑셀파일 다운로드

#### 크롬 드라이버 객체 생성

In [4]:
driver = webdriver.Chrome(service=service)

#### 웹페이지 로드

In [5]:
url = 'https://www.opinet.co.kr/user/main/mainView.do'

In [6]:
driver.get(url)

#### [싼주유소찾기]-[지역] 페이지로 이동

- 자바스트립트 코드 실행으로 통해서 이동

In [7]:
driver.execute_script('goSubPage(0,0,99)')

#### 시 이름 찾기

In [8]:
xpath = '//*[@id="SIDO_NM0"]'
sido_lists = driver.find_element(By.XPATH, xpath)
sido_opt= sido_lists.find_elements(By.TAG_NAME, 'option')
sido_names = [opt.get_attribute('value') for opt in sido_opt]
sido_names

['',
 '서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

#### 구 이름 찾기

In [9]:
xpath = '//*[@id="SIGUNGU_NM0"]'
gu_lists = driver.find_element(By.XPATH, xpath)
gu_opt= gu_lists.find_elements(By.TAG_NAME, 'option')
gu_names = [opt.get_attribute('value') for opt in gu_opt]
gu_names

['',
 '강릉시',
 '고성군',
 '동해시',
 '삼척시',
 '속초시',
 '양구군',
 '양양군',
 '영월군',
 '원주시',
 '인제군',
 '정선군',
 '철원군',
 '춘천시',
 '태백시',
 '평창군',
 '홍천군',
 '화천군',
 '횡성군']

#### '강남구' 주유소 지정

In [10]:
elementsido = driver.find_element(By.ID, 'SIDO_NM0')
elementsido.send_keys('서울시')

In [11]:
elementgu = driver.find_element(By.ID, 'SIGUNGU_NM0')
elementgu.send_keys('강남구')

#### '조회' 버튼 클릭

In [12]:
xpath = '//*[@id="searRgSelect"]'
driver.find_element(By.XPATH, xpath).click()

#### 조회된 주유소 '엑셀저장' 버튼 클릭

In [13]:
xpath = '//*[@id="glopopd_excel"]'
driver.find_element(By.XPATH, xpath).click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="glopopd_excel"]"}
  (Session info: chrome=124.0.6367.62); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00CAC113+48259]
	(No symbol) [0x00C3CA41]
	(No symbol) [0x00B30A17]
	(No symbol) [0x00B70BED]
	(No symbol) [0x00B70C9B]
	(No symbol) [0x00BABC12]
	(No symbol) [0x00B90DE4]
	(No symbol) [0x00BA9B9C]
	(No symbol) [0x00B90B36]
	(No symbol) [0x00B6570D]
	(No symbol) [0x00B662CD]
	GetHandleVerifier [0x00F66533+2908323]
	GetHandleVerifier [0x00FA3B4B+3159739]
	GetHandleVerifier [0x00D4505B+674763]
	GetHandleVerifier [0x00D4B21C+699788]
	(No symbol) [0x00C46244]
	(No symbol) [0x00C42298]
	(No symbol) [0x00C4242C]
	(No symbol) [0x00C34BB0]
	BaseThreadInitThunk [0x76DAFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77B27C5E+286]
	RtlGetAppContainerNa

=> '지역_위치별(주유소).xls'파일 다운로드 폴더에 저장됨

### 25개 구 주유소 엑셀파일 다운로드

- 함수로 작성하여 실행

In [ ]:
import time

In [ ]:
# 시도 이름 찾기
xpath = '//*[@id="SIDO_NM0"]'
sido_lists = driver.find_element(By.XPATH, xpath)
sido_opt= sido_lists.find_elements(By.TAG_NAME, 'option')
sido_names = [opt.get_attribute('value') for opt in sido_opt]

# 구 이름 찾기
xpath = '//*[@id="SIGUNGU_NM0"]'
gu_lists = driver.find_element(By.XPATH, xpath)
gu_opt= gu_lists.find_elements(By.TAG_NAME, 'option')
gu_names = [opt.get_attribute('value') for opt in gu_opt]

# 시도 이름 지정
elementsido = driver.find_element(By.ID, 'SIDO_NM0')
elementsido.send_keys('서울시')


# 시군구 휘발유 가격 엑셀파일 다운로드 함수
def search_download_sigungu(sigungu):
    # 구 이름 지정
    elementgu = driver.find_element(By.ID, 'SIGUNGU_NM0')
    elementgu.send_keys(sigungu)
    time.sleep(2)

# 조회버튼 클릭
    xpath = '//*[@id="searRgSelect"]'
    driver.find_element(By.XPATH, xpath).click()
    time.sleep(2)

# 엑셀저장 버튼 클릭
    xpath = '//*[@id="glopopd_excel"]'
    driver.find_element(By.XPATH, xpath).click()
    time.sleep(2)
    
# 입력한 시도의 모든 시군구 휘발유 가격 엑셀파일 다운로드 함수
def search_download_sido(sido):
    # 시도 이름 선택
    elementsido = driver.find_element(By.ID, 'SIDO_NM0')
    elementsido.send_keys(sido)
    time.sleep(2)
    
    # 시군구 이름 찾기
    xpath = '//*[@id="SIGUNGU_NM0"]'
    gu_lists = driver.find_element(By.XPATH, xpath)
    gu_opt= gu_lists.find_elements(By.TAG_NAME, 'option')
    gu_names = [opt.get_attribute('value') for opt in gu_opt][1:]

    # 시군구 조회후 엑셀다운
    for sigungu in gu_names:
        search_download_sigungu(sigungu)

# 전체 시도별 주유소 가격 다운로드
def search_download_all():
    # 시도이름 찾기
    xpath = '//*[@id="SIDO_NM0"]'
    sido_lists = driver.find_element(By.XPATH, xpath)
    sido_opt= sido_lists.find_elements(By.TAG_NAME, 'option')
    sido_names = [opt.get_attribute('value') for opt in sido_opt]
    
    # 시도별 주유소 가격 다운로드
    for sido in sido_names:
        search_download_sido(sido)

In [ ]:
search_download_sido('서울특별시')

### 엑셀 파일 읽어오기

In [ ]:
# glob 함수: 파일 경로 및 이름을 모아서 리스트에 저장
from glob import glob

In [ ]:
import os
cpath = os.getcwd()
cpath

In [ ]:
path = 'C:/Users/gillhk/Downloads/'

In [ ]:
filename = '지역_위치별*.xls'
files = glob(path+filename)
len(files)

In [ ]:
!pip install xlrd

In [ ]:
station = pd.read_excel(files[0], header=2)
station

In [ ]:
type(station)

#### 데이터 합치기

In [ ]:
tmp = []
for file in files:
    station = pd.read_excel(file, header=2)
    tmp.append(station)
stations = pd.concat(tmp)

In [ ]:
stations

In [ ]:
stations.info()

In [ ]:
stations.to_csv('주유소.csv')

In [ ]:
driver.close()

-----